In [ ]:
!unzip archive.zip -d monkey

In [ ]:
!pip install tensorflow

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/monkey/training/training"
val_dir   = "/content/monkey/validation/validation"

train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=20,width_shift_range=0.1, height_shift_range=0.1,
    horizontal_flip=True, zoom_range=0.1)
val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(train_dir,target_size=(128,128),batch_size=32,class_mode='categorical')
val_gen = val_datagen.flow_from_directory(val_dir,target_size=(128,128),batch_size=32,class_mode='categorical')


Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [5]:
from tensorflow.keras import layers, models

# CNN
scratch_cnn = models.Sequential([
    layers.Input(shape=(128, 128, 3)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax'),
])

scratch_cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy']
)

#train & validate
history_scratch = scratch_cnn.fit(train_gen,validation_data=val_gen,epochs=10)

#final validation accuracy
val_loss, val_acc = scratch_cnn.evaluate(val_gen)
print(f"Scratch CNN Validation accuracy: {val_acc:.3f}")


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.0894 - loss: 2.3703 - val_accuracy: 0.2096 - val_loss: 2.1162
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.2726 - loss: 2.0860 - val_accuracy: 0.3493 - val_loss: 1.8441
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.3453 - loss: 1.8671 - val_accuracy: 0.4853 - val_loss: 1.5210
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.4156 - loss: 1.6794 - val_accuracy: 0.5074 - val_loss: 1.3853
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.4036 - loss: 1.6049 - val_accuracy: 0.5588 - val_loss: 1.2834
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.4739 - loss: 1.4346 - val_accuracy: 0.5772 - val_loss: 1.2253
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.4967 - loss: 1.3591 - val_accuracy: 0.5625 - val_loss: 1.2774
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.4941 - loss: 1.3806 - val_accuracy: 0.6140 - val_loss:

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
import tensorflow as tf

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory("/content/monkey/training/training",target_size=(128, 128),batch_size=32,
    class_mode='categorical'
)
val_gen = val_datagen.flow_from_directory(
    "/content/monkey/validation/validation",
    target_size=(128, 128),batch_size=32,
    class_mode='categorical'
)

#load the MobileNetV2 base as feature extractor
base_model = MobileNetV2(
    input_shape=(128, 128, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

#
inputs = layers.Input(shape=(128, 128, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = models.Model(inputs, outputs)

#head only
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history_head = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

# evaluate head-only performance
loss_head, acc_head = model.evaluate(val_gen)
print(f"Head-only Validation Accuracy: {acc_head:.3f}")

#unfreezign top convolutional layers for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5
)

# final evaluation after fine-tuning
loss_fine, acc_fine = model.evaluate(val_gen)
print(f"Fine-tuned Validation Accuracy: {acc_fine:.3f}")


Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.
Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.5253 - loss: 1.4756 - val_accuracy: 0.9375 - val_loss: 0.2457
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.8968 - loss: 0.3079 - val_accuracy: 0.9559 - val_loss: 0.1843
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9211 - loss: 0.2111 - val_accuracy: 0.9449 - val_loss: 0.1671
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9483 - loss: 0.1582 - val_accuracy: 0.9375 - val_loss: 0.2056
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9677 - loss: 0.0996 - val_accuracy: 0.9522 - val_loss: 0.1579
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9704 - loss: 0.0813 - val_accuracy: 0.9485 - val_loss: 0.1907
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.9741 - loss: 0.0696 - val_accuracy: 0.9375 - val_loss: 0.2181
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━